![MLU Logo](../data/MLU_Logo.png)

# <a name="0">Machine Learning Accelerator - Natural Language Processing - Lecture 2</a>

## Final Project: Tree-based Model for the IMDB Movie Review Dataset

For the final project, build a K Nearest Neighbors model to predict the sentiment (positive or negative) of movie reviews. The dataset is originally hosted here: http://ai.stanford.edu/~amaas/data/sentiment/

Use the notebooks from the class and implement the model, train and test with the corresponding datasets.

You can follow these steps:
1. Read training-test data (Given)
2. Train a classifier (Implement)
3. Make predictions on your test dataset (Implement)

You can use the __LogisticRegression__ from here: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html  
You can use the __DecisionTreeClassifier__ from here: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html  
You can use the __RandomForestClassifier__ from here: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html  
You can use the __GridSearchCV__ from here: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html  
You can use the __RandomizedSearchCV__ from here: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html


## 1. Reading the dataset

We will use the __pandas__ library to read our dataset.

#### __Training data:__
Let's read our training data. Here, we have the text and label fields. Labe is 1 for positive reviews and 0 for negative reviews.

In [1]:
import pandas as pd

train_df = pd.read_csv('../data/final_project/imdb_train.csv', header=0)
train_df.head()

,text,label
0,This movie makes me want to throw up every tim...,0
1,Listening to the director's commentary confirm...,0
2,One of the best Tarzan films is also one of it...,1
3,Valentine is now one of my favorite slasher fi...,1
4,No mention if Ann Rivers Siddons adapted the m...,0


#### __Test data:__

In [2]:
import pandas as pd

test_df = pd.read_csv('../data/final_project/imdb_test.csv', header=0)
test_df.head()

,text,label
0,What I hoped for (or even expected) was the we...,0
1,Garden State must rate amongst the most contri...,0
2,There is a lot wrong with this film. I will no...,1
3,"To qualify my use of ""realistic"" in the summar...",1
4,Dirty War is absolutely one of the best politi...,1


## 2. Train a Classifier

You can work with these models:
* __LogisticRegression__ from here: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html  
* __DecisionTreeClassifier__ from here: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html  
* __RandomForestClassifier__ from here: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html  
* __GridSearchCV__ from here: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html  
* __RandomizedSearchCV__ from here: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

In [3]:
#Exlore distribution of classes
print(train_df['label'].value_counts())
print(test_df['label'].value_counts())

#Explore NaNs
print(train_df.isnull().values.any())
print(test_df.isnull().values.any())

0    12500
1    12500
Name: label, dtype: int64
0    12500
1    12500
Name: label, dtype: int64
False
False


In [4]:
#Download relevant nltk packages
import nltk, re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anshbordia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anshbordia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anshbordia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
#Stop words removal
#Not all stop words are bad in this scenario. We will retain some helpful words as shown below
from nltk.corpus import stopwords
excluding = ['against', 'not', 'don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't",
             'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 
             'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't",
             'needn', "needn't",'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', 
             "weren't", 'won', "won't", 'wouldn', "wouldn't"]

accepted_stopwords = list(set(stopwords.words('english')).symmetric_difference(set(excluding)))

In [6]:
#Setup tokenizer and lemmatizer for preprocessing
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer()

In [7]:
"""
Convert to lower case
Remove trailing white space
Remove intermediate extra white space
Remove HTML tags
Lemmatize individual words
"""
def preprocess(text):
    processed_sentences = []
    temp = ""
    for i in range(0, len(text)):
        temp = text[i]
        temp = temp.lower()
        temp = temp.strip()
        temp = re.sub('\s+', ' ', temp)
        temp = re.compile('<.*?>').sub('', temp)
        
        selected_words = []
        for word in word_tokenize(temp):
            if(word not in accepted_stopwords):
                selected_words.append(lemmatizer.lemmatize(word))
        
        processed_sentences.append(" ".join(selected_words))
        temp = ""
    return processed_sentences

In [8]:
train_X = preprocess(train_df['text'].values)
test_X = preprocess(test_df['text'].values)

In [9]:
train_y = train_df['label'].values
test_y = test_df['label'].values

In [10]:
# Build TF-IDF features to be fed into model
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 5000)
train_X = vectorizer.fit_transform(train_X)

In [11]:
vectorizer.vocabulary_

{'movie': 2928,
 'make': 2719,
 'want': 4817,
 'throw': 4503,
 'every': 1568,
 'time': 4518,
 'see': 3893,
 'take': 4394,
 'first': 1758,
 'plot': 3324,
 'leave': 2567,
 'sea': 3878,
 'daughter': 1162,
 'go': 1950,
 'character': 779,
 'give': 1938,
 'new': 3008,
 'animal': 261,
 'name': 2965,
 'crappy': 1070,
 'animation': 263,
 'biggest': 503,
 'suck': 4307,
 'factor': 1650,
 'possible': 3369,
 'get': 1927,
 'little': 2631,
 'basically': 433,
 'copy': 1035,
 'll': 2636,
 'horror': 2184,
 'step': 4223,
 'people': 3241,
 '1st': 44,
 'prince': 3425,
 'eric': 1543,
 'still': 4233,
 'line': 2617,
 'whole': 4886,
 'annoying': 271,
 'not': 3049,
 'spirit': 4161,
 'saw': 3844,
 'fact': 1649,
 'exact': 1578,
 'opposite': 3128,
 'le': 2554,
 'cute': 1129,
 'convincing': 1027,
 'joke': 2425,
 'funny': 1878,
 'anymore': 281,
 'kid': 2475,
 'talk': 4400,
 'dumb': 1412,
 'voice': 4788,
 'king': 2485,
 'probably': 3436,
 'best': 490,
 'love': 2674,
 'melody': 2806,
 'appear': 295,
 'like': 2607,
 's

In [28]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0, n_estimators = 100)
score = cross_val_score(rf, train_X, train_y , cv=5).mean()
score

0.8431200000000001

In [30]:
#Random Search
from sklearn.model_selection import RandomizedSearchCV
params = dict(n_estimators = [150, 200, 300], oob_score = ['True', 'False'])
rf_rs = RandomizedSearchCV(rf, params, random_state=0, n_iter = 4)
models = rf_rs.fit(train_X, train_y)


In [31]:
models.best_params_

{'oob_score': 'False', 'n_estimators': 300}

In [60]:
models.best_score_

0.84864

In [61]:
#Evaluate performance on test set using RF
from sklearn.metrics import classification_report
print('*************************Random Forest*********************************')
print(classification_report(test_y, rf_rs.predict(vectorizer.transform(test_X))))

*************************Random Forest*********************************
              precision    recall  f1-score   support

           0       0.84      0.86      0.85     12500
           1       0.86      0.84      0.85     12500

    accuracy                           0.85     25000
   macro avg       0.85      0.85      0.85     25000
weighted avg       0.85      0.85      0.85     25000

